Descargamos el dataset en un pandas dataframe

In [21]:
import pandas as pd
import os
import time
file = "Online Retail.xlsx"
url = "https://github.com/Alf-caput/LAB02_ReglasYPatronesSecuenciales/raw/main/Online%20Retail.xlsx"
if os.path.exists(file):
    target = file
else:
    target = url
df = pd.read_excel(target)

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


Con **.info** podemos ver el número de filas, columnas y los tipos almacenados.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   541909 non-null  int64         
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 33.1+ MB


Observamos 541909 entradas y 8 columnas.  
Buscando documentación en kaggle:  
Se trata de un dataset que contiene las transacciones que han ocurrido entre 01/12/2009 y 09/12/2011 en UK de una compañía que vende principalmente regalos de ocasión, adicionalmente se especifica que muchos de los compradores son mayoristas.  
Columnas del dataset:
- **InvoiceNo:**  
    - Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- **StockCode:**
    - Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- **Description:**
    - Product (item) name. Nominal.
- **Quantity:** 
    - The quantities of each product (item) per transaction. Numeric.
- **InvoiceDate:**
    - Invoice date and time. Numeric. The day and time when a transaction was generated.
- **UnitPrice**:
    - Unit price. Numeric. Product price per unit in sterling (Â£).
- **CustomerID:**
    - Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- **Country:**
    - Country name. Nominal. The name of the country where a customer resides.

Viendo los tipos que devuelve **.info** vemos que pandas nos ha guardado Quantity, UnitPrice y CustomerID como tipos numéricos, mientras que el resto excepto InvoiceDate (tipo fecha) son de tipo object, que es el tipo por defecto que usa pandas para strings.

Con **.isna** podemos ver cuantos valores na tiene en total y por columna el dataframe.  
(De **.info** también se puede deducir pero es menos intuitivo)

In [23]:
print(f'Número total de NA en el dataframe: {(col_na:=df.isna().sum()).sum()}')
pd.DataFrame({'Valores NA': col_na})

Número total de NA en el dataframe: 1454


,Valores NA
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


Encontramos pocos valores NA, que podríamos completar si conocemos su StockCode, no obstante estudiamos un poco más el dataset antes de tomar una acción en este aspecto.  
Con **.describe** podemos ver el rango de valores que pueden tomar diferentes columnas (omitimos la columna "count" y transponemos por legibilidad).

In [24]:
(
    df
    .describe()
    .drop('count')
    .T
)

,mean,min,25%,50%,75%,max,std
Quantity,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,15287.518434,12346.0,14367.0,15287.0,16255.0,18287.0,1484.746041


Vemos que existen valores que no nos interesan para el problema que queremos resolver, existen cantidades y precio de unidades negativas ("Quantity", "UnitPrice") que pueden deberse a devoluciones u otro tipo de gestión en el producto.  

In [25]:
df_negs = (
    df
    .loc[(df.loc[:, 'Quantity'] <= 0) | (df.loc[:, 'UnitPrice'] <= 0), :]
)
print(f"Entradas con cantidad o precio unidad negativas (o cero): {df_negs.shape[0]}")

negs_ratio = round(len(df_negs)/len(df), 2)
print(f"Ratio: {negs_ratio}")

df_negs.head()

Entradas con cantidad o precio unidad negativas (o cero): 11805
Ratio: 0.02


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom


Tenemos más de 11_000 entradas, que suponen un 2% del dataset.  
Como no nos interesan podemos eliminar esas entradas.  

In [26]:
df_std = df.drop(df_negs.index)
df_std.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [27]:
import numpy as np
df_invoice = df_std.groupby(['CustomerID', 'InvoiceDate', 'InvoiceNo'])['StockCode'].apply(lambda x: np.array(x)).reset_index()
df_invoice.sort_values(by='CustomerID')
df_invoice.head()

,CustomerID,InvoiceDate,InvoiceNo,StockCode
0,12346,2011-01-18 10:01:00,541431,[23166]
1,12347,2010-12-07 14:57:00,537626,"[85116, 22375, 71477, 22492, 22771, 22772, 227..."
2,12347,2011-01-26 14:30:00,542237,"[84625A, 84625C, 85116, 20719, 22375, 22376, 2..."
3,12347,2011-04-07 10:43:00,549222,"[22376, 22374, 22371, 22375, 20665, 23076, 217..."
4,12347,2011-06-09 13:01:00,556201,"[23084, 23162, 23171, 23172, 23170, 23173, 231..."


La celda de abajo no es muy explicativa y seguro que es posible simplificarla, pero basicamente consigue agregar una columna InvoiceGroup que servirá para agrupar las transacciones que disten 5 o menos minutos, esto es útil si un cliente aparece en dos transacciones a la misma hora o si aparece en dos transacciones que disten menos de 5 minutos.

In [28]:
max_distance = pd.Timedelta(minutes=5)
df_invoice_groupmin = pd.DataFrame(columns=df_invoice.columns)
for customer in df_invoice.loc[:, 'CustomerID'].unique():
    df_invoice_groupmin_customer = df_invoice.loc[df_invoice.loc[:, 'CustomerID'] == customer, :].copy(deep=True)
    df_invoice_groupmin_customer.loc[:, 'InvoiceGroup'] = (
        df_invoice_groupmin_customer
        .sort_values('InvoiceDate')
        .groupby('CustomerID')['InvoiceDate']
        .diff()
        .gt(max_distance)
        .cumsum()
    )
    df_invoice_groupmin = pd.concat([df_invoice_groupmin, df_invoice_groupmin_customer])

# Example customer=13047
df_invoice_groupmin.loc[df_invoice_groupmin.loc[:, 'CustomerID'] == 13047, :]

C:\Users\alfre\AppData\Local\Temp\ipykernel_13180\2065503899.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_invoice_groupmin = pd.concat([df_invoice_groupmin, df_invoice_groupmin_customer])


,CustomerID,InvoiceDate,InvoiceNo,StockCode,InvoiceGroup
2402,13047,2010-12-01 08:34:00,536367,"[84879, 22745, 22748, 22749, 22310, 84969, 226...",0.0
2403,13047,2010-12-01 08:34:00,536368,"[22960, 22913, 22912, 22914]",0.0
2404,13047,2010-12-01 08:35:00,536369,[21756],0.0
2405,13047,2011-02-03 13:06:00,543113,"[16161P, 22766, 22708, 23231, 85015, 85016, 22...",1.0
2406,13047,2011-03-30 13:28:00,548331,"[22960, 22722, 22720, 20972, 22274, 84969, 226...",2.0
2407,13047,2011-05-04 14:11:00,551846,"[23152, 23182, 23161, 23160, 23163, 23162, 231...",3.0
2408,13047,2011-06-19 13:56:00,557299,"[23092, 23112, 23110, 23111, 23118, 23093, 230...",4.0
2409,13047,2011-08-04 17:58:00,562437,"[22623, 16161P, 21754, 21755, 21901, 21900, 48...",5.0
2410,13047,2011-10-14 11:15:00,571185,"[22739, 23435, 22960, 22722, 22720, 20972, 226...",6.0
2411,13047,2011-11-08 12:06:00,575043,[M],7.0


Gracias a crear la columna anterior podemos agrupar las transacciones con mismo CustomerID y mismo InvoiceGroup.  
Teniendo en cuenta que hay que añadir los StockCodes a la transacción en la fecha más reciente.

In [29]:
df_invoice_simple = (
    df_invoice_groupmin
    .groupby(['CustomerID', 'InvoiceGroup'])
    .apply(lambda x: pd.Series({
        'StockCode': np.concatenate(x['StockCode'].values),
        'InvoiceDate': x.loc[x['InvoiceDate'].idxmax(), 'InvoiceDate'],
        'InvoiceNo': x.loc[x['InvoiceDate'].idxmax(), 'InvoiceNo']
})).reset_index())
df_invoice_simple.head()

C:\Users\alfre\AppData\Local\Temp\ipykernel_13180\3967295710.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


,CustomerID,InvoiceGroup,StockCode,InvoiceDate,InvoiceNo
0,12346,0.0,[23166],2011-01-18 10:01:00,541431
1,12347,0.0,"[85116, 22375, 71477, 22492, 22771, 22772, 227...",2010-12-07 14:57:00,537626
2,12347,1.0,"[84625A, 84625C, 85116, 20719, 22375, 22376, 2...",2011-01-26 14:30:00,542237
3,12347,2.0,"[22376, 22374, 22371, 22375, 20665, 23076, 217...",2011-04-07 10:43:00,549222
4,12347,3.0,"[23084, 23162, 23171, 23172, 23170, 23173, 231...",2011-06-09 13:01:00,556201


Por simplificar podemos eliminar la columna Invoice (mirando InvoiceDate sabremos que transacción ha ocurrido antes).  
Y también podemos convertir el número de transacción en índice de fila.

In [30]:
df_invoice_simple.drop(columns='InvoiceGroup', inplace=True)
df_invoice_simple.set_index('InvoiceNo', inplace=True)
df_invoice_simple.head()

,CustomerID,StockCode,InvoiceDate
InvoiceNo,,,
541431,12346,[23166],2011-01-18 10:01:00
537626,12347,"[85116, 22375, 71477, 22492, 22771, 22772, 227...",2010-12-07 14:57:00
542237,12347,"[84625A, 84625C, 85116, 20719, 22375, 22376, 2...",2011-01-26 14:30:00
549222,12347,"[22376, 22374, 22371, 22375, 20665, 23076, 217...",2011-04-07 10:43:00
556201,12347,"[23084, 23162, 23171, 23172, 23170, 23173, 231...",2011-06-09 13:01:00


In [31]:
len(df_invoice_simple['CustomerID'].unique())

4338

In [32]:
len(df_invoice_simple)

18395

In [33]:
df_invoice_simple.loc[df_invoice_simple.loc[:, 'CustomerID'] == 12347, :]

,CustomerID,StockCode,InvoiceDate
InvoiceNo,,,
537626,12347,"[85116, 22375, 71477, 22492, 22771, 22772, 227...",2010-12-07 14:57:00
542237,12347,"[84625A, 84625C, 85116, 20719, 22375, 22376, 2...",2011-01-26 14:30:00
549222,12347,"[22376, 22374, 22371, 22375, 20665, 23076, 217...",2011-04-07 10:43:00
556201,12347,"[23084, 23162, 23171, 23172, 23170, 23173, 231...",2011-06-09 13:01:00
562032,12347,"[23308, 23297, 22375, 22374, 22376, 22371, 223...",2011-08-02 08:48:00
573511,12347,"[23480, 21265, 21636, 22372, 22375, 22371, 223...",2011-10-31 12:25:00
581180,12347,"[23497, 23552, 21064, 84625A, 21731, 23084, 20...",2011-12-07 15:52:00


In [47]:
from gsppy.gsp import GSP
transactions = [
        ['Bread', 'Milk'],
        ['Bread', 'Diaper', 'Beer', 'Eggs'],
        ['Milk', 'Diaper', 'Beer', 'Coke'],
        ['Bread', 'Milk', 'Diaper', 'Beer'],
        ['Bread', 'Milk', 'Diaper', 'Coke']
]
result = GSP(transactions).search(0.3)
result

[{('Bread',): 4, ('Diaper',): 4, ('Beer',): 3, ('Milk',): 4, ('Coke',): 2},
 {('Diaper', 'Beer'): 3, ('Bread', 'Milk'): 3, ('Milk', 'Diaper'): 3},
 {('Bread', 'Milk', 'Diaper'): 2, ('Milk', 'Diaper', 'Beer'): 2}]

In [46]:
from gsppy.gsp import GSP
transactions = [
        [('Bread', 'Milk')],
        ['Bread', 'Diaper', 'Beer', 'Eggs'],
        ['Milk', 'Diaper', 'Beer', 'Coke'],
        ['Bread', 'Milk', 'Diaper', 'Beer'],
        ['Bread', 'Milk', 'Diaper', 'Coke']
]
result = GSP(transactions).search(0.3)
result

[{('Bread',): 3, ('Beer',): 3, ('Diaper',): 4, ('Coke',): 2, ('Milk',): 3},
 {('Diaper', 'Beer'): 3, ('Milk', 'Diaper'): 3, ('Bread', 'Milk'): 2},
 {('Bread', 'Milk', 'Diaper'): 2, ('Milk', 'Diaper', 'Beer'): 2}]

In [35]:
# from gsppy.gsp import GSP
# # muñeco1 = df_invoice_simple.loc[df_invoice_simple.loc[:, 'CustomerID'] == 12347, :]['StockCode']
# # gsp1 = GSP(muñeco1, ).search(0.25)
# customers = df_invoice_simple.loc[:, 'CustomerID'].unique()
# rules = []
# for customer in customers:
#     df_customer = df_invoice_simple.loc[df_invoice_simple.loc[:, 'CustomerID'] == customer, :]['StockCode']
#     gsp_customer = GSP(df_customer).search(0.25)
#     # print(gsp_customer)
#     if not gsp_customer:
#         print(gsp_customer)
#         continue
#     if len(gsp_customer)>1:
#         print(len(gsp_customer))
#     for res in gsp_customer[1:]:
#         for rule in res.keys():
#             # print(rule)
#             # if len(rule) > 1:
#             rules.append(rule)
#             print(rule)
#             # break
#     if rules:
#         print(len(rules))
#         break
# print(rules)

In [36]:
print(rules)

[('79066K', '79191C')]
